In [89]:
import sys
! pip install kaggle seaborn pandas numpy matplotlib
!{sys.executable} -m pip install kaggle seaborn pandas numpy matplotlib

In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

df = pd.read_csv("customer_shopping_data.csv")


# TRANSFORMACION DE DATOS
Convertiremos los datos en dimensiones que podremos utilizar facilmente

In [91]:
import string
def convertirIndice(idxString):
    lista_letras = list(string.ascii_lowercase)
    idxString = str(idxString)
    nuevo_idx = ""
    for char in idxString:
        if char.lower() in lista_letras:
            nuevo_idx += str(lista_letras.index(char.lower()))
        else:
            nuevo_idx += char
    return int(nuevo_idx)

In [92]:
df['customer_id'] = [convertirIndice(x) for x in df['customer_id']]
df['invoice_no'] = [convertirIndice(x) for x in df['invoice_no']]
df.head()

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,8138884,2241288,Female,28,Clothing,5,1500.40,Credit Card,5/8/2022,Kanyon
1,8317333,2111565,Male,21,Shoes,3,1800.51,Debit Card,12/12/2021,Forum Istanbul
2,8127801,2266599,Male,20,Clothing,1,300.08,Cash,9/11/2021,Metrocity
3,8173702,2988172,Female,66,Shoes,5,3000.85,Credit Card,16/05/2021,Metropol AVM
4,8337046,2189076,Female,53,Books,4,60.60,Cash,24/10/2021,Kanyon


In [93]:
# Crear la tabla DimCliente
DimCliente = df[['customer_id', 'gender', 'age']].drop_duplicates().reset_index(drop=True)
DimCliente.rename(columns={'customer_id': 'IDCliente', 'gender': 'Genero', 'age': 'Edad'}, inplace=True)
DimCliente.head()


,IDCliente,Genero,Edad
0,2241288,Female,28
1,2111565,Male,21
2,2266599,Male,20
3,2988172,Female,66
4,2189076,Female,53


In [94]:
# Crear la tabla DimProducto
DimProducto = df[['category']].drop_duplicates().reset_index(drop=True)
DimProducto['IDProducto'] = DimProducto.index + 1
DimProducto.rename(columns={'category': 'Categoria'}, inplace=True)

DimProducto.head()

,Categoria,IDProducto
0,Clothing,1
1,Shoes,2
2,Books,3
3,Cosmetics,4
4,Food & Beverage,5


In [95]:
# Crear la tabla DimFecha
df['invoice_date'] = pd.to_datetime(df['invoice_date'], dayfirst=True, errors='coerce')
DimFecha = df[['invoice_date']].drop_duplicates().reset_index(drop=True)
DimFecha['IDFecha'] = DimFecha.index + 1
DimFecha['Año'] = DimFecha['invoice_date'].dt.year
DimFecha['Mes'] = DimFecha['invoice_date'].dt.month
DimFecha['DiaSemana'] = DimFecha['invoice_date'].dt.day_name()
DimFecha.rename(columns={'invoice_date': 'FechaFactura'}, inplace=True)

#DimFecha.sort_values(['Año', 'Mes', 'DiaSemana'])
DimFecha.head()

,FechaFactura,IDFecha,Año,Mes,DiaSemana
0,2022-08-05,1,2022,8,Friday
1,2021-12-12,2,2021,12,Sunday
2,2021-11-09,3,2021,11,Tuesday
3,2021-05-16,4,2021,5,Sunday
4,2021-10-24,5,2021,10,Sunday


In [96]:
# Crear la tabla DimTienda
DimTienda = df[['shopping_mall']].drop_duplicates().reset_index(drop=True)
DimTienda['IDTienda'] = DimTienda.index + 1
DimTienda.rename(columns={'shopping_mall': 'CentroComercial'}, inplace=True)
DimTienda.head()

,CentroComercial,IDTienda
0,Kanyon,1
1,Forum Istanbul,2
2,Metrocity,3
3,Metropol AVM,4
4,Istinye Park,5


In [97]:
# Fusionar con cada tabla de dimensiones
Ventas = df.merge(DimCliente, left_on='customer_id', right_on='IDCliente')\
           .merge(DimProducto, left_on='category', right_on='Categoria')\
           .merge(DimFecha, left_on='invoice_date', right_on='FechaFactura')\
           .merge(DimTienda, left_on='shopping_mall', right_on='CentroComercial')

# Seleccionar solo las columnas necesarias
Ventas = Ventas[['invoice_no', 'IDCliente', 'IDProducto', 'IDFecha', 'IDTienda', 'quantity', 'price', 'payment_method']]
Ventas.rename(columns={'invoice_no': 'NumFacturaNominal', 'quantity': 'Cantidad', 'price': 'Precio', 'payment_method': 'MetodoPago'}, inplace=True)
Ventas.head()

,NumFacturaNominal,IDCliente,IDProducto,IDFecha,IDTienda,Cantidad,Precio,MetodoPago
0,8138884,2241288,1,1,1,5,1500.40,Credit Card
1,8317333,2111565,2,2,2,3,1800.51,Debit Card
2,8127801,2266599,1,3,3,1,300.08,Cash
3,8173702,2988172,2,4,4,5,3000.85,Credit Card
4,8337046,2189076,3,5,1,4,60.60,Cash


# Gráficas del Dataset
Empezamos a visualizar los datos

In [103]:
#----------Dispersion de las ventas por mes------------
# Asegúrate de que ambas columnas son de tipo datetime
Ventas['invoice_date'] = pd.to_datetime(Ventas['invoice_date'], dayfirst=True, errors='coerce')
DimFecha['FechaFactura'] = pd.to_datetime(DimFecha['FechaFactura'], dayfirst=True, errors='coerce')

# Hacemos el merge usando la fecha
df_ventas = Ventas.merge(DimFecha[['FechaFactura','IDFecha','Año','Mes','DiaSemana']],
                         left_on='invoice_date',
                         right_on='FechaFactura',
                         how='left')
'''plt.figure(figsize=(10, 6))
sns.scatterplot(x=df_ventas['Año', 'Mes'], y=df_ventas['Precio'] * df_ventas['Cantidad'])
plt.show()'''


KeyError: 'invoice_date'